In [1]:
import pandas as pd
import folium
import requests
import urllib.parse

In [2]:
df = pd.read_hdf('outputs/budzety_clean_topics_bert.h5')

## Zdobywamy współrzędne geograficzne miast

In [3]:
cities = df['miasto'].unique().tolist()

# Mamy w sumie 66 miast
len(cities)

66

In [4]:
# Zapytanie do servera Nominatim

cities_dict = dict()
for city in cities:
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(city) +'?format=json'
    response = requests.get(url).json()
    lat = response[0]['lat']
    lon = response[0]['lon']
    cities_dict[city] = (lat, lon)


In [5]:
# Odnalezione współrzędne
cities_dict

{'GDYNIA': ('54.5164982', '18.5402738'),
 'SUWAŁKI': ('54.102712499999996', '22.931878217494493'),
 'GDAŃSK': ('54.36119285', '18.62860883362069'),
 'SOPOT': ('54.44190905', '18.54309444570447'),
 'BIELSKO-BIAŁA': ('49.822118', '19.0448936'),
 'BYTOM': ('50.3657432', '18.87153251609449'),
 'CHORZÓW': ('50.288556299999996', '18.97044253093511'),
 'CZĘSTOCHOWA': ('50.8089997', '19.1244089281233'),
 'DĄBROWA GÓRNICZA': ('50.3308692', '19.2078918'),
 'GLIWICE': ('50.294113', '18.6657306'),
 'JASTRZĘBIE ZDRÓJ': ('49.9519085', '18.6023614'),
 'JAWORZNO': ('50.203439', '19.2721559'),
 'KATOWICE': ('50.2598987', '19.0215852'),
 'PIEKARY': ('51.39338', '19.5203'),
 'MYSŁOWICE': ('50.2096211', '19.132165997410812'),
 'RUDA ŚLĄSKA': ('50.270303', '18.864375646334572'),
 'RYBNIK': ('50.0955793', '18.5419933'),
 'SIEMIANOWICE ŚLĄSKIE': ('50.31285075', '19.017785857354603'),
 'SOSNOWIEC': ('50.2780834', '19.1342944'),
 'ŚWIĘTOCHŁOWICE': ('50.2989445', '18.907466206416952'),
 'TYCHY': ('50.114397', '

## Sprawdzamy poprawność

In [6]:
# Sprawdźmy wizualnie czy nie ma błędnie odczytanych współrzędnych i faktycznie 
# wszystkie znalezione miasta leżą w Polsce

my_map = folium.Map(
    location=[52.2319581, 21.0067249],
    zoom_start=7
)

for key, values in cities_dict.items():
    folium.Marker(
        location=[float(values[0]), float(values[1])],
        tooltip=key
    ).add_to(my_map)

In [7]:
my_map

## Tworzymy nowy dataframe ze współrzędnymi miast

In [8]:
df_sample = df[['rok', 'miasto', 'topics', 'topics_desc']]
df_sample

,rok,miasto,topics,topics_desc
0,2020,GDYNIA,-1,"[chodnik, park, ulica, budowa, warsztat, konce..."
1,2020,GDYNIA,3,"[park, ulica, kieszonkowy, dziecko, szach, spo..."
2,2020,GDYNIA,1,"[fontanna, woda, toaleta, wodny, park, łazienk..."
3,2020,GDYNIA,8,"[drzewo, ogród, łąka, kwiat, trawnik, kwiatowy..."
4,2020,GDYNIA,-1,"[chodnik, park, ulica, budowa, warsztat, konce..."
...,...,...,...,...
23025,2020,KRAKÓW,13,"[zielony, zieleń, zielono, ulica, zieleniec, w..."
23026,2020,KRAKÓW,2,"[parking, parkingowy, miejsce, bezdomność, szk..."
23027,2020,KRAKÓW,11,"[powietrze, smog, monitoring, siłownia, szkoła..."
23028,2020,KRAKÓW,15,"[łódź, plaża, plażowy, łódzki, morski, kajakow..."


In [9]:
df_sample['coords'] = df_sample['miasto'].map(cities_dict)
df_sample['lat'] = df_sample['coords'].map(lambda x: float(x[0]))
df_sample['lon'] = df_sample['coords'].map(lambda x: float(x[1]))

C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [10]:
df_sample

,rok,miasto,topics,topics_desc,coords,lat,lon
0,2020,GDYNIA,-1,"[chodnik, park, ulica, budowa, warsztat, konce...","(54.5164982, 18.5402738)",54.516498,18.540274
1,2020,GDYNIA,3,"[park, ulica, kieszonkowy, dziecko, szach, spo...","(54.5164982, 18.5402738)",54.516498,18.540274
2,2020,GDYNIA,1,"[fontanna, woda, toaleta, wodny, park, łazienk...","(54.5164982, 18.5402738)",54.516498,18.540274
3,2020,GDYNIA,8,"[drzewo, ogród, łąka, kwiat, trawnik, kwiatowy...","(54.5164982, 18.5402738)",54.516498,18.540274
4,2020,GDYNIA,-1,"[chodnik, park, ulica, budowa, warsztat, konce...","(54.5164982, 18.5402738)",54.516498,18.540274
...,...,...,...,...,...,...,...
23025,2020,KRAKÓW,13,"[zielony, zieleń, zielono, ulica, zieleniec, w...","(50.0619474, 19.9368564)",50.061947,19.936856
23026,2020,KRAKÓW,2,"[parking, parkingowy, miejsce, bezdomność, szk...","(50.0619474, 19.9368564)",50.061947,19.936856
23027,2020,KRAKÓW,11,"[powietrze, smog, monitoring, siłownia, szkoła...","(50.0619474, 19.9368564)",50.061947,19.936856
23028,2020,KRAKÓW,15,"[łódź, plaża, plażowy, łódzki, morski, kajakow...","(50.0619474, 19.9368564)",50.061947,19.936856


## Sprawdźmy najpopularniejszy temat w danym mieście

In [11]:
df_sample.groupby(['miasto']).agg(pd.Series.mode)['topics'].values

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, 6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, array([-1,  0], dtype=int64), -1,
       -1, -1, -1, -1, 3], dtype=object)

In [340]:
# Zwracane jest bardzo dużo nieprzypisanych tematów (-1) oraz w jednym przypadku modą
# są dwie wartości. Zastosujmy sięc inne podejście.

In [11]:
# Zwróć najpopularniejszy temat, który nie jest -1
def most_common_topics(city: str) -> int:
  result = df_sample[(df_sample['miasto'] == city)]['topics'].value_counts()[0:2]
  if result.index[0] == -1:
    return result.index[1]
  else:
    return result.index[0]


## Budujemy dataframe'a, z którego będą pobierane informacje do mapy

In [12]:
df_cities = pd.DataFrame({'cities': cities})

In [13]:
df_cities['topics'] = df_cities['cities'].map(most_common_topics)

In [14]:
df_cities

,cities,topics
0,GDYNIA,0
1,SUWAŁKI,1
2,GDAŃSK,0
3,SOPOT,1
4,BIELSKO-BIAŁA,2
...,...,...
61,RZESZÓW,4
62,TARNOBRZEG,0
63,BIAŁYSTOK,4
64,ŁOMŻA,0


In [15]:
# Robimy mapowanie numerów tematów z opisami
df_cities['topics_desc'] = df_cities['topics'].map(df_sample.drop_duplicates('topics').set_index('topics')['topics_desc'])

In [16]:
# Robimy mapowanie nazw miast z ich współrzędnymi
df_cities['coords'] = df_cities['cities'].map(cities_dict)
df_cities['lat'] = df_cities['coords'].map(lambda x: float(x[0]))
df_cities['lon'] = df_cities['coords'].map(lambda x: float(x[1]))

In [17]:
# Opisy tematów zapisujemy jako str
df_cities['desc'] = df_cities['topics_desc'].map(lambda x: ' '.join(x))

In [18]:
# Usuwamy niepotrzebne kolumny
if 'coords' in df_cities.columns:
    del df_cities['coords']
    del df_cities['topics_desc']

In [19]:
df_cities

,cities,topics,lat,lon,desc
0,GDYNIA,0,54.516498,18.540274,parking dzielnica rower parkingowy miejski mia...
1,SUWAŁKI,1,54.102712,22.931878,fontanna woda toaleta wodny park łazienka kana...
2,GDAŃSK,0,54.361193,18.628609,parking dzielnica rower parkingowy miejski mia...
3,SOPOT,1,54.441909,18.543094,fontanna woda toaleta wodny park łazienka kana...
4,BIELSKO-BIAŁA,2,49.822118,19.044894,parking parkingowy miejsce bezdomność szkoła j...
...,...,...,...,...,...
61,RZESZÓW,4,50.037453,22.004717,szkoła przedszkole szkolny przedszkolny edukac...
62,TARNOBRZEG,0,50.592422,21.696435,parking dzielnica rower parkingowy miejski mia...
63,BIAŁYSTOK,4,53.127505,23.147051,szkoła przedszkole szkolny przedszkolny edukac...
64,ŁOMŻA,0,53.182431,22.052184,parking dzielnica rower parkingowy miejski mia...


## Dodajemy do dataframe'a kolory i ikony

In [20]:
topic_nums = df_cities['topics'].unique().tolist()
colors = ('lightgreen', 'cadetblue', 'gray', 'orange', 'darkred', 'darkgreen', 'red', 'black', 'lightgray', 'green', 'lightred', 'purple', 'lightblue')
icons = ('road','bicycle', 'tint', 'car','pagelines', 'building', 'compass', 'book', 'save', 'certificate', 'edit', 'globe', 'file')

topics_dict = dict()
for n, elem in enumerate(topic_nums):
    topics_dict[elem] = (colors[n], icons[n])

In [21]:
topics_dict

{0: ('lightgreen', 'road'),
 1: ('cadetblue', 'bicycle'),
 2: ('gray', 'tint'),
 4: ('orange', 'car'),
 10: ('darkred', 'pagelines'),
 17: ('darkgreen', 'building'),
 5: ('red', 'compass'),
 14: ('black', 'book'),
 3: ('lightgray', 'save'),
 6: ('green', 'certificate'),
 11: ('lightred', 'edit'),
 13: ('purple', 'globe'),
 7: ('lightblue', 'file')}

In [22]:
df_cities['colors'] = df_cities['topics'].map(topics_dict).apply(lambda x: x[0])
df_cities['icons'] = df_cities['topics'].map(topics_dict).apply(lambda x: x[1])

In [23]:
df_cities

,cities,topics,lat,lon,desc,colors,icons
0,GDYNIA,0,54.516498,18.540274,parking dzielnica rower parkingowy miejski mia...,lightgreen,road
1,SUWAŁKI,1,54.102712,22.931878,fontanna woda toaleta wodny park łazienka kana...,cadetblue,bicycle
2,GDAŃSK,0,54.361193,18.628609,parking dzielnica rower parkingowy miejski mia...,lightgreen,road
3,SOPOT,1,54.441909,18.543094,fontanna woda toaleta wodny park łazienka kana...,cadetblue,bicycle
4,BIELSKO-BIAŁA,2,49.822118,19.044894,parking parkingowy miejsce bezdomność szkoła j...,gray,tint
...,...,...,...,...,...,...,...
61,RZESZÓW,4,50.037453,22.004717,szkoła przedszkole szkolny przedszkolny edukac...,orange,car
62,TARNOBRZEG,0,50.592422,21.696435,parking dzielnica rower parkingowy miejski mia...,lightgreen,road
63,BIAŁYSTOK,4,53.127505,23.147051,szkoła przedszkole szkolny przedszkolny edukac...,orange,car
64,ŁOMŻA,0,53.182431,22.052184,parking dzielnica rower parkingowy miejski mia...,lightgreen,road


In [24]:
# Zapisujemy do pliku
df_cities.to_hdf('outputs/folium_vis_with_topics_bert.h5', key='df_cities')

In [31]:
# Generujemy mapę
my_map = folium.Map(
    location=[52.2319581, 21.0067249],
    zoom_start=7
)

# Odkomentować, jeżeli chcemy dodać różne style do naszej mapy

# folium.raster_layers.TileLayer('Open Street Map').add_to(my_map)
# folium.raster_layers.TileLayer('Stamen Terrain').add_to(my_map)
# folium.raster_layers.TileLayer('Stamen Toner').add_to(my_map)
# folium.raster_layers.TileLayer('Stamen Watercolor').add_to(my_map)
# folium.raster_layers.TileLayer('CartoDB Positron').add_to(my_map)
# folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(my_map)
# folium.LayerControl().add_to(my_map)


for _, row in df_cities.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        tooltip=f'<b>{row.cities}</b><br><br>Temat nr {row.topics}<br> Kliknij po opis',
        popup=f'<b>Najpopularniejszy temat - nr {row.topics}:</b><br>{row.desc}',
        icon=folium.Icon(color=row.colors, prefix='fa',icon=row.icons)
    ).add_to(my_map)

In [32]:
my_map

In [33]:
# Zapisujemy mapę jako html
# Zapisana mapa nie wymaga dostępu do utworzonego dataframe'a. Informacje o 
# punktach, opisy, ikony, kolory itp. są umieszczone bezpośrednio w pliku html.
my_map.save('outputs/sample_map.html')